# Python WAT

por Flávio Juvenal  
Consultor em Vinta Software (http://www.vinta.com.br)

## Identidade
quando um objeto em Python é (`is`) outro?

In [1]:
# Hmm 🤔
a = 256
b = 256
a is b

True

In [2]:
# WTF 😧
a = 257
b = 257
a is b

False

Python tem uma cache de inteiros de [-5, 256], o que faz com que `a` e `b` sejam referências para o mesmo objeto caso seus valores sejam iguais e estejam dentro do intervalo [-5, 256].

Note que isto é um detalhe da implementação CPython e pode ser diferente em outras implementações. 

Fonte: https://docs.python.org/3/c-api/long.html#c.PyLong_FromLong

In [3]:
# Hmm 🤔
!cat int_example.py

a = 257
b = 257
print(a is b)


In [4]:
# WTF 😧
!python int_example.py

True


In [5]:
# Hmm 🤔
me = {'name': 'Fulano', 'age': 30}
people = [me] * 3
people

[{'age': 30, 'name': 'Fulano'},
 {'age': 30, 'name': 'Fulano'},
 {'age': 30, 'name': 'Fulano'}]

In [6]:
# WTF 😧
people[0]['name'] = 'Sicrano'
people

[{'age': 30, 'name': 'Sicrano'},
 {'age': 30, 'name': 'Sicrano'},
 {'age': 30, 'name': 'Sicrano'}]

`[me] * 3` é equivalente a `[me, me, me]`, ou seja, 3 referências para o mesmo objeto

In [7]:
# Hmm 🤔
line = [' '] * 3
line

[' ', ' ', ' ']

In [8]:
# Hmm 🤔
table = [line] * 3
table

[[' ', ' ', ' '], [' ', ' ', ' '], [' ', ' ', ' ']]

In [9]:
def print_table():
    for l in table:
        print(l)
print_table()

[' ', ' ', ' ']
[' ', ' ', ' ']
[' ', ' ', ' ']


In [10]:
# WTF 😧
table[0][0] = 'X'
print_table()

['X', ' ', ' ']
['X', ' ', ' ']
['X', ' ', ' ']


In [11]:
# Ohh 😮
table[0] is table[1] and table[1] is table[2]

True

In [12]:
# Ohh 😮
id(table[0]) == id(table[1]) == id(table[2])

True

In [13]:
# WTF 😧
table[2][0] = 'O'
print_table()

['O', ' ', ' ']
['O', ' ', ' ']
['O', ' ', ' ']


In [14]:
# Nice 🙂
table = [[' ' for j in range(3)] for i in range(3)]
print_table()

[' ', ' ', ' ']
[' ', ' ', ' ']
[' ', ' ', ' ']


In [15]:
# Nice 🙂
table[2][0] = 'O'
print_table()

[' ', ' ', ' ']
[' ', ' ', ' ']
['O', ' ', ' ']


In [16]:
# Hmm 🤔
table = [[' '] * 3 for i in range(3)]
print_table()

[' ', ' ', ' ']
[' ', ' ', ' ']
[' ', ' ', ' ']


In [17]:
# WTF 😧
table = [[{}] * 3 for i in range(3)]
table[0][0]['name'] = 'Fulano'
print_table()

[{'name': 'Fulano'}, {'name': 'Fulano'}, {'name': 'Fulano'}]
[{}, {}, {}]
[{}, {}, {}]


Resumindo: para tabelas, faça dois `for`s

## Variáveis de classe
você está usando variáveis de classe corretamente?

In [18]:
# Hmm 🤔
class Dog:
    tricks = []

    def __init__(self, name):
        self.name = name
    
    def add_trick(self, trick):
        self.tricks.append(trick)

    def print_tricks(self):
        print(self.name, ' tricks:')
        for trick in self.tricks:
            print(trick)

In [19]:
# Hmm 🤔
teddy = Dog("Teddy")
teddy.add_trick("roll over")
teddy.print_tricks()

Teddy  tricks:
roll over


In [20]:
# WTF 😧
kika = Dog("Kika")
kika.add_trick("play dead")
kika.print_tricks()

Kika  tricks:
roll over
play dead


In [21]:
# WTF 😧
teddy.print_tricks()

Teddy  tricks:
roll over
play dead


In [22]:
# Nice 🙂
class Dog:
    def __init__(self):
        self.tricks = []

    # ...

Cuidado com a diferença entre variáveis de classe e de instância

Mais info: https://docs.python.org/3/tutorial/classes.html#class-and-instance-variables
Exemplo de bug real: https://github.com/allisson/django-pagseguro2/pull/6

In [23]:
# Hmm 🤔
class A:
    x = 1

class B(A):
    pass

class C(A):
    pass

print(A.x, B.x, C.x)

1 1 1


In [24]:
# Hmm 🤔
B.x = 2
print(A.x, B.x, C.x)

1 2 1


In [25]:
# WTF 😧
A.x = 3
print(A.x, B.x, C.x)

3 2 3


Quando um atributo ou método não é encontrado em uma classe, ele é procurado nas suas classes mães, seguindo o MRO (Method Resolution Order)

In [28]:
# WTF 😧
class A:
    x = 1

class B(A):
    pass

a_instance = A()
a_instance.x = 2
print("A.x", A.x)
print("B.x", B.x)

A.x 1
B.x 1


Se uma variável é atribuída através da instância, ela é uma variável de instância. Se é atribuída através da classe, ela é uma variável de classe.

Mais info:  
https://www.toptal.com/python/top-10-mistakes-that-python-programmers-make#common-mistake-2-using-class-variables-incorrectly  
https://www.toptal.com/python/python-class-attributes-an-overly-thorough-guide#handling-assignment